In [6]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [11]:
import os
homeDir = os.getcwd()+"/"
fileDir = homeDir+"Analysis/"
workDir = "/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/"

In [13]:
%%R -i fileDir -i workDir 
#devtools::install_github(repo="knausb/vcfR")
#install.packages('doParallel')
library(vcfR)
library(doParallel)
#setwd("/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/")
dir.create(fileDir)
setwd(workDir)

# VCF-------------------
vcfDir <- "ensemble/vcf/"
filenames <- dir(vcfDir,"ensemble.snpEff.vcf.gz")
#filenames <- filenames[!grepl(".tbi",filenames)]

vcfRead <- function(filename,vcfDir){
  tumorID <- sub("-ensemble.snpEff.vcf.gz","",filename)
  tmp <- vcfR::read.vcfR(paste0(vcfDir,filename), verbose=FALSE)
  tmpVcf <- tmp@fix
  tmpGt <- tmp@gt
  if(length(colnames(tmpGt))==3){
    if(!grepl("_N\\b",colnames(tmpGt)[3])){ #CHECK THE NAME OF TUMOR AND NORMAL SAMPLES IN VCF!
      tmpGt <- tmpGt[,c(1,3,2)]
    }
  }
  info <- tmpVcf[,"INFO"]
  func <- sapply(info,function(x){
    temp <- unlist(strsplit(x,","))
    paste0(unique(na.omit(unlist(sapply(temp,function(x) unlist(strsplit(x,"|",fixed=TRUE))[2])))),collapse=",")
  })
  func <- unname(func)
  callers <- sapply(info,function(x){
    temp <- unlist(strsplit(x,";"))
    sub("CALLERS=","",temp[grepl("CALLERS=",temp)])
  })
  callers <- unname(callers)
  id <- paste0(tumorID,"-",tmpVcf[,1],":",tmpVcf[,2])
  if(length(colnames(tmpGt))==3){
    tmp <- data.frame(tmpVcf[,c(1,2,4,5,7)],tmpGt,tumorID,info,func,callers,id,stringsAsFactors=FALSE)
  } else {
    tmp <- data.frame(tmpVcf[,c(1,2,4,5,7)],tmpGt,normal=NA,tumorID,info,func,callers,id,stringsAsFactors=FALSE)
  }
  colnames(tmp) <- c("CHROM","POS","REF","ALT","FILTER","FORMAT","Tumor","Normal","Tumor_ID","snpEff","snpEff_func","caller","id")
  rownames(tmp) <- NULL
  return(tmp)
}

vcf <- lapply(filenames,function(x) vcfRead(x,vcfDir))
vcf <- do.call(rbind,vcf)
nrow(vcf) #306237


R[write to console]: 
   *****       ***   vcfR   ***       *****
   This is vcfR 1.12.0 
     browseVignettes('vcfR') # Documentation
     citation('vcfR') # Citation
   *****       *****      *****       *****


R[write to console]: Loading required package: foreach

R[write to console]: Loading required package: iterators

R[write to console]: Loading required package: parallel



[1] 306237


## CORRECT VCF

In [14]:
%%R 
# CORRECT FOR NA ALT
table(is.na(vcf$ALT))
# vcf <- vcf[!is.na(vcf$ALT),]

# CORRECT FOR MULTIPLE ALT
table(grepl(",",vcf$ALT))
tmp <- sapply(vcf$ALT,function(x) unlist(strsplit(x,",",fixed=TRUE))[1])
vcf$ALT <- tmp

# # CORRECT FOR NA LINES
# sum(is.na(vcf))

## SELECT CODING/SPLICING

In [15]:
%%R 
# SELECT CODING/SPLICING
tmp <- unique(vcf$snpEff_func)
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,",")))))
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,"&")))))
tmp <- tmp[!grepl("ENS",tmp)]
tmp <- tmp[!grepl("=",tmp)]
sort(tmp)
sel <- c("5_prime_UTR_premature_start_codon_gain_variant","5_prime_UTR_truncation", #NEW
         "3_prime_UTR_variant", "5_prime_UTR_variant", # Added
         "bidirectional_gene_fusion","conservative_inframe_deletion","conservative_inframe_insertion",
         "disruptive_inframe_deletion","disruptive_inframe_insertion",
         "exon_loss_variant","initiator_codon_variant","gene_fusion",
         "frameshift_variant","missense_variant","splice_acceptor_variant",
         "splice_donor_variant","splice_region_variant","start_lost","stop_gained",
         "stop_lost","stop_retained_variant")
setdiff(tmp,sel)
setdiff(sel,tmp)
sel <- sapply(sel,function(x) grep(x,vcf$snpEff_func))
sel <- unique(unlist(sel))
sel <- sort(sel)
sum(is.na(sel))
nrow(vcf) #227168
rownames(vcf) <- NULL
length(sel) #36938
vcf <- vcf[sel,]
sort(table(vcf$Tumor_ID))

callerN <- sapply(vcf$caller,function(x) length(unlist(strsplit(x,",",fixed=TRUE))))
callerN <- unname(callerN)
vcf <- data.frame(vcf,callerN,stringsAsFactors = FALSE)
table(vcf$callerN)
#   1     2     3     4     5     6 
# 29690  3753   704   328   438  2025  
rownames(vcf) <- NULL
#----------------------------------------------

save(vcf,file = paste(fileDir,"vcf.rda",sep=""))

## COUNTS and ALLELE FREQUENCY

In [18]:
%%R 
# COUNTS and ALLELE FREQUENCY -----------------

countsVcf <- function(vcf,i){
  Tmp <- rep(NA,4)
  names(Tmp) <- c("t_ref_count","t_alt_count","n_ref_count","n_alt_count")
  vcfLine <- as.character(vcf[i,])
  names(vcfLine) <- colnames(vcf)
  caller <- unlist(strsplit(vcfLine["caller"],",",fixed=TRUE))[1]
  if(caller=="varscan"){
    tmp <- unlist(strsplit(vcfLine["Tumor"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    Tmp["t_alt_count"] <- as.numeric(tmp["AD"])
    Tmp["t_ref_count"] <- as.numeric(tmp["RD"])
    tmp <- unlist(strsplit(vcfLine["Normal"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    Tmp["n_alt_count"] <- as.numeric(tmp["AD"])
    Tmp["n_ref_count"] <- as.numeric(tmp["RD"])
  } else {
    tmp <- unlist(strsplit(vcfLine["Tumor"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    if(tmp["AD"]!=".") Tmp[c("t_ref_count","t_alt_count")] <- as.numeric(unlist(strsplit(tmp["AD"],","))[1:2])
    if(!is.na(vcfLine["Normal"])) {
      tmp <- unlist(strsplit(vcfLine["Normal"],":"))
      names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))[1:length(tmp)]
      if(tmp["AD"]!=".") Tmp[c("n_ref_count","n_alt_count")] <- as.numeric(unlist(strsplit(tmp["AD"],","))[1:2])
    }
  }
  return(Tmp)
}

cl <- makePSOCKcluster(60,outfile="")
registerDoParallel(cl)
tmp <- foreach(i = 1:nrow(vcf)) %dopar% {
  tmp <- countsVcf(vcf,i)
  return(tmp)
}
stopCluster(cl)

tmp <- do.call(rbind,tmp)
dim(tmp) #36938     4

counts <- cbind(tmp[,1:2],NA,NA,tmp[,3:4],NA,NA)
colnames(counts) <- c("t_ref_count","t_alt_count","t_depth","t_vaf","n_ref_count","n_alt_count","n_depth","n_vaf")
counts[,"t_depth"] <- counts[,"t_alt_count"] + counts[,"t_ref_count"]
counts[,"t_vaf"] <- round(counts[,"t_alt_count"]/counts[,"t_depth"],2)
counts[,"n_depth"] <- counts[,"n_alt_count"] + counts[,"n_ref_count"]
counts[,"n_vaf"] <- round(counts[,"n_alt_count"]/counts[,"n_depth"],2)
rownames(counts) <- NULL

#View(counts[sample(1:nrow(counts),100),])

vcf <- data.frame(vcf, counts, stringsAsFactors = FALSE)

table(cond <- is.na(vcf$n_depth)&!is.na(vcf$Normal))
# FALSE  TRUE 
# 27894  9044 
# table(cond <- is.na(vcf$n_depth))

tmp <- vcf[cond,]

# table(tmp$caller)
# freebayes 
# 9044

vcf <- vcf[!cond,]

rownames(vcf) <- NULL
nrow(vcf) #27894
rm(counts)

#----------------------------------------------
# save(vcf,file = "Analysis/vcf.rda")

## FP FILTER

In [19]:
fpfilterDir = fileDir + "fpfilter/"

In [20]:
%%R -i fpfilterDir
# FP FILTER
#system("mkdir Analysis/fpfilter")
#setwd("Analysis/fpfilter/")
dir.create(fpfilterDir)
setwd(fpfilterDir)

vcf <- data.frame(vcf,fpFid="",stringsAsFactors=FALSE)
patients <- unique(vcf$Tumor_ID)
for(i in 1:length(patients)){
  print(patients[i])
  name <- patients[i]
  var <- vcf[vcf$Tumor_ID==name,1:4]
  var[nchar(var$ALT)>1,"ALT"] <- paste0("+",substring(var[nchar(var$ALT)>1,"ALT"],2))
  var[nchar(var$REF)>1,"POS"] <- as.numeric(var[nchar(var$REF)>1,"POS"])+1
  var[nchar(var$REF)>1,"ALT"] <- paste0("-",substring(var[nchar(var$REF)>1,"REF"],2))
  var[nchar(var$REF)>1,"REF"] <- substring(var[nchar(var$REF)>1,"REF"],2,2)
  vcf[vcf$Tumor_ID==name,"fpFid"] <- paste0(name,"-",var[,1],":",var[,2],"-",var[,4])
  write.table(var,file=paste0(name,".var"),sep="\t",quote=FALSE,row.names=FALSE,col.names=FALSE)
  # n <- ceiling(nrow(var)/60)
  # system(paste0("split -l ",n," --additional-suffix .var ",paste0(name,".var")))
  loc <- paste(var$CHROM,var$POS,var$POS)
  write.table(loc,file=paste0(name,".loc"),quote=FALSE,row.names=FALSE,col.names=FALSE)
  # system(paste0("split -l ",n," --additional-suffix .loc ",paste0(name,".loc")))
}

# UNIX 
# bamreadcount.sh
# fpfilter.sh

failfiles <- dir(path=".",pattern = "fpfilter.fail")
FPfail <- NULL
for(i in 1:length(failfiles)){
  print(i)
  patient <- unlist(strsplit(failfiles[i],".",fixed=TRUE))[1]
  tmp <- readLines(failfiles[i])
  tmp <- sapply(tmp,function(x){
    Tmp <- unlist(strsplit(x,"\t"))
    length(Tmp) <- 9
    Tmp
  })
  tmp <- t(tmp)
  failID <- paste0(patient,"-",tmp[,1],":",tmp[,2],"-",tmp[,4])
  tmp <- cbind(failID,tmp[,9])
  rownames(tmp) <- failID
  FPfail <- rbind(FPfail,tmp)
}

passfiles <- dir(path=".",pattern = "fpfilter.pass")
FPpass <- NULL
for(i in 1:length(passfiles)){
  print(i)
  patient <- unlist(strsplit(passfiles[i],".",fixed=TRUE))[1]
  tmp <- read.delim(passfiles[i],header=FALSE,as.is=TRUE)
  tmp <- paste0(patient,"-",tmp[,1],":",tmp[,2],"-",tmp[,4])
  FPpass <- c(FPpass,tmp)
}

length(FPpass) #8967
nrow(FPfail) #18886

length(FPpass)+nrow(FPfail) #27853

all <- c(FPpass,FPfail[,1])
table(vcf$fpFid%in%all)
View(vcf[!vcf$fpFid%in%all,])

vcf <- data.frame(vcf,FPfilter=".",FPfail=".",stringsAsFactors=FALSE)
vcf[vcf$fpFid%in%FPpass,"FPfilter"] <- "PASS"
vcf[vcf$fpFid%in%rownames(FPfail),"FPfilter"] <- "FAIL"
vcf[vcf$FPfilter=="FAIL","FPfail"] <- FPfail[vcf[vcf$FPfilter=="FAIL","fpFid"],2]

vcf[is.na(vcf$FPfail),"FPfail"] <- ""
vcf[vcf$FPfail=="","FPfilter"] <- "no_readcounts" # alt absent in the bam readcount file
vcf[vcf$FPfail=="","FPfail"] <- "."
vcf[vcf$FPfilter==".","FPfilter"] <- "no_readcounts"# alt = 0 in the bam readcount file
table(vcf$FPfilter)
# FAIL no_readcounts          PASS 
# 15577          3350          8967  
rm(FPpass,FPfail,all)

[1] "A1783"
[1] "A9155"
[1] "A9878"
[1] "AT2550"
[1] "AT2822"
[1] "AT4415"
[1] "AT4808"
[1] "C0288"
[1] "C0334"
[1] "C1572"
[1] "C8448"
[1] "s1_3076_D"
[1] "s1_3295_D"
[1] "s1_8588_D"
[1] "s2_6088_D"
[1] "s2_6339_D"
[1] "s2_8811_D"
[1] "s3_5832_D"
[1] "s3_7299_D"
[1] "s3_7979_D"
[1] 1


R[write to console]: Error in file(con, "r") : cannot open the connection

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In file(con, "r") :
R[write to console]:  cannot open file 'NA': No such file or directory




Error in file(con, "r") : cannot open the connection
